In [1]:
%cd ..

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
import pandas as pd

from scripts.utils_latest import *
pd.set_option('display.max_columns', None)


# import CLASH and make predictions

In [3]:
clash_df = clash()
# df = find_matches_with_rnaduplex(clash_df)
# df.to_csv("results/0_rnaduplex_predictions_raw.csv", index=False)
df = pd.read_csv("results/0_rnaduplex_predictions_raw.csv")

# data generation & extraction to csv

In [4]:
# adding helper columns from clash
df["id"] = clash_df["id"]
df["clash_mrna_start"] = clash_df["true_start"]
df["clash_mrna_end"] = clash_df["true_end"]
df["true_seed_type"] = clash_df["true_seed_type"]
df["true_num_basepairs"] = clash_df["num_basepairs"]
df["true_seed_basepairs"] = clash_df["seed_basepairs"]
df["true_energy"] = clash_df["folding_energy"]
df["mirna_sequence"] = clash_df["mirna_sequence"]
df["mirna_accession"] = clash_df["mirna_accession"]
df["enst"] = clash_df["enst"]
df["ensg"] = clash_df["ensg"]
df["full_mrna_seq"] = clash_df["mrna_sequence"]


In [5]:
# feature generation
df = generate_alignment_string_from_dot_bracket(df)
df = generate_match_count_columns(df)

# adding bp difference columns
df["bp_difference"] = df["pred_num_basepairs"] - df["true_num_basepairs"]
df["seed_bp_difference"] = df["pred_seed_basepairs"] - df["true_seed_basepairs"]

In [6]:
df.head()

,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,id,clash_mrna_start,clash_mrna_end,true_seed_type,...,mirna_sequence,mirna_accession,enst,ensg,full_mrna_seq,alignment_string,pred_num_basepairs,pred_seed_basepairs,bp_difference,seed_bp_difference
0,23,46,-27.7,1,21,)))))))))))))))))))).,0727A-1038930_1,1791,1890,noncanonical_seed,...,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340828,ENSG00000113328,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,1111111111111111111100,20,6,0,0
1,12,31,-26.2,1,19,))))))))).)))))))).,L1HS-1112536_1,3857,3928,9-mer,...,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000343455,ENSG00000100697,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,1111111110111111110000,17,6,0,0
2,4,25,-23.6,1,22,.)))))).))))))))))))).,L2HS-818542_2,2385,2434,noncanonical_seed,...,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000436639,ENSG00000080546,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,0111111011111111111110,19,6,0,0
3,4,25,-23.8,1,22,)))))))...)))))))))))),L2HS-1161339_2,6570,6623,noncanonical_seed,...,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000282516,ENSG00000164190,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,1111111000111111111111,19,6,1,0
4,3,25,-24.0,1,22,))))))))))))))...)))).,L2-407944_2,1164,1208,noncanonical_seed,...,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340139,ENSG00000138785,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,1111111111111100011110,18,6,0,0


In [7]:
def generate_mre_region(df):
    df["mirna_length"] = df["mirna_sequence"].str.len()
    

In [8]:
# MRE region

df["mirna_length"] = df["mirna_sequence"].str.len()
df["mre_start"] = df["mrna_start"] - df["mirna_start"] + 1
df["mre_end"] = df["mre_start"] + df["mirna_length"] - 1
df.drop("mirna_length", axis=1, inplace=True)
# mre_start has values less than 1. convert them all into 1s.
# df['mre_start'] = df['mre_start'].apply(lambda x: max(x, 1))

# generating mre region sequence
df["full_mrna_seq"] = clash_df["mrna_sequence"]


def slice_column(row):
    return row['full_mrna_seq'][row['mre_start']-1:row['mre_end']]


df['mre_region'] = df.apply(slice_column, axis=1)
# df.drop("full_mrna_seq", axis=1, inplace=True)

In [12]:
# writing to csv
df.to_csv("results/1_generate_data.csv", index=False)


############################